In [1]:
# determine the optimal pre-match forecast, given elo_diff and s_elo_diff
# step 3: calculate glicko and s_glicko for each player and do the same

In [4]:
import sys
sys.path.insert(0,'/Users/jacobgollub/Desktop/college (current)/research/pbp_explorations/scripts/sackmann')
import tennisGameProbability,tennisMatchProbability,tennisSetProbability,tennisTiebreakProbability
from tennisMatchProbability import matchProb
from helper_functions import validate_results
import pandas as pd
import numpy as np
from sklearn.metrics import log_loss,accuracy_score

# can test this on our subset of 10,000 matches as well as all matches in the database:
#df = pd.read_csv('../my_data/elo_pbp_with_surface_8_27.csv')
df = pd.read_csv('../my_data/elo_atp_matches_21st_century_9_2.csv')
del df['Unnamed: 0']

# currently looking at 2014 tour-level matches, excluding Davis Cup
#df = df[df['match_year']==2014].reset_index(drop=True)
#df = df[df['match_year'].isin([2013,2014,2015,2016])]
df = df[df['tourney_name']!='Davis Cup'].reset_index(drop=True)
df.head(2)

,tourney_id,tourney_name,tourney_date,match_year,p0_name,p1_name,p0_elo,p1_elo,elo_diff,p0_s_elo,...,tourney_stats,best_of,score,pbp,winner,match_id,p0_s_kls,p1_s_kls,p0_s_kls_JS,p1_s_kls_JS
0,2000-339,Adelaide,2000-01-03,2000,Thomas Enqvist,Arnaud Clement,1989.436848,1741.104988,248.33186,1939.223943,...,0.631615,3,6-3 6-4,None,1,0,0.590656,0.536052,0.592050,0.539075
1,2000-339,Adelaide,2000-01-03,2000,Jens Knippschild,Roger Federer,1755.204529,1703.745759,51.45877,1597.559723,...,0.631615,3,6-1 6-4,None,0,1,0.600610,0.577408,0.601691,0.580165


In [1]:
#df.loc[39289]

In [14]:
# generate match probabilities and z-scores for Klaassen method, with and w/o JS estimators
df['match_prob_kls'] = [matchProb(row['p0_s_kls'],1-row['p1_s_kls']) for i,row in df.iterrows()]
df['match_prob_kls_JS'] = [matchProb(row['p0_s_kls_JS'],1-row['p1_s_kls_JS']) for i,row in df.iterrows()]

from scipy.stats import norm
for col in ['match_prob_kls','match_prob_kls_JS']:
    df[col.replace('prob','z')] = norm.ppf(df[col])
print df.shape

(46222, 48)


In [17]:
print 'elo 538 baseline: ',  sum((df['elo_diff_538']>0) == df['winner'])/float(len(df))
print log_loss(df['winner'],[(1+10**(diff/-400.))**-1 for diff in df['elo_diff_538']])
print log_loss(df['winner'],[(1+10**(diff/-400.))**-1 for diff in df['s_elo_diff_538']])


print 'surface elo 538 baseline: ', sum((df['s_elo_diff_538']>0) == df['winner'])/float(len(df))

current = (0,0)
for weight in np.arange(11)*.1:
    accuracy = sum(((1-weight)*df['elo_diff_538']+weight*df['s_elo_diff_538']>0) == df['winner'])/float(len(df))
    if accuracy > current[1]:
        current=weight,accuracy
print 'best s_elo weight: ',current[0],'accuracy =',current[1]

elo 538 baseline:  0.677664315694
0.618563286636
0.612628547157
surface elo 538 baseline:  0.67850806975
best s_elo weight:  0.5 accuracy = 0.684133096794


In [53]:
# print 'elo baseline: ',  sum((df['elo_diff']>0) == df['winner'])/float(len(df))
# print 'surface elo baseline: ', sum((df['s_elo_diff']>0) == df['winner'])/float(len(df))

# current = (0,0)
# for weight in np.arange(10)*.1:
#     accuracy = sum(((1-weight)*df['elo_diff']+weight*df['s_elo_diff']>0) == df['winner'])/float(len(df))
#     if accuracy > current[1]:
#         current=weight,accuracy
# print 'best s_elo weight: ',current[0],'accuracy =',current[1]

elo baseline:  0.689199920588
surface elo baseline:  0.683541790748
best s_elo weight:  0.3 accuracy = 0.694361723248


In [ ]:
# seems worth trying methods to alter match_prob_kls with elo

In [18]:
cols = [['elo_diff_538'],['elo_diff','s_elo_diff'],['elo_diff_538','s_elo_diff_538'],\
        ['elo_diff','s_elo_diff','match_z_kls'],\
        ['elo_diff_538','s_elo_diff_538','match_z_kls']]
n_splits = 5
validate_results(df,cols,n_splits)

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

In [6]:
# now, create win_prob columns for elo

In [9]:
# logistic regression on z scores of our probabilities AND elo_diff/s_elo_diff
from scipy.stats import norm
df_all = pd.read_csv('../my_data/feature_df_pbp3_8_23_alphas.csv')
del df_all['Unnamed: 0']
for col in ['sackmann_prob','klaassen_prob','klaassen_prob_a_400']:
    df_all[col.replace('prob','z')] = norm.ppf(df_all[col])
df_all.loc[df_all['sackmann_z']>10,'sackmann_z'] = 10
df_all.loc[df_all['klaassen_z']>10,'klaassen_z'] = 10
df_all.loc[df_all['klaassen_z_a_400']>10,'klaassen_z_a_400'] = 10

In [11]:
from sklearn import linear_model
from sklearn.model_selection import KFold
from sklearn.metrics import log_loss,accuracy_score

# also will want to try ensemble methods at some point...
kfold = KFold(n_splits=5,shuffle=True)
scores = [],[]
cols = ['elo_diff','s_elo_diff','klaassen_z_a_400']
for train_ind,test_ind in kfold.split(df_all):
    lm = linear_model.LogisticRegression(fit_intercept = True)
    train_df,test_df = df_all.loc[train_ind],df_all.loc[test_ind]
    lm.fit(train_df[cols].values.reshape([len(train_df),len(cols)]),train_df['winner'])
    y_preds = lm.predict(test_df[cols].values.reshape([len(test_df),len(cols)]))
    y_probs = lm.predict_proba(test_df[cols].values.reshape([len(test_df),len(cols)]))
    accuracy = accuracy_score(test_df['winner'],y_preds)
    loss = log_loss(test_df['winner'],y_probs,labels=[0,1])
    scores[0].append(accuracy);scores[1].append(loss)
#print '% s_elo used in lm fit: ',lm.coef_[0][1]/(lm.coef_[0][0]+lm.coef_[0][1])
print 'coefficients: ', lm.coef_[0]
print 'accuracy: ', np.mean(scores[0])
print 'loss: ', np.mean(scores[1])

coefficients:  [ -5.39480879e-04   1.72440109e-03   1.71413858e+00]
accuracy:  0.778048803942
loss:  0.456072287662


In [ ]:
# perform same analysis across entire match dataset, over the decades

In [161]:
# one-time fix to transfer over the elo_diff's
# import pandas as pd
# import numpy as np

# df_og = pd.read_csv('../my_data/elo_pbp_with_surface_8_22.csv')
# del df_og['Unnamed: 0']
# df = pd.read_csv('../my_data/feature_df_pbp3_8_23_alphas.csv')
# del df['Unnamed: 0']

# elo_dict = dict(zip(range(len(df_og)),df_og['elo_diff']))
# s_elo_dict = dict(zip(range(len(df_og)),df_og['s_elo_diff']))

# df['elo_diff'] = [elo_dict[m_id] for m_id in df['match_id']]
# df['s_elo_diff'] = [s_elo_dict[m_id] for m_id in df['match_id']]
# df.to_csv('../my_data/feature_df_pbp3_8_23_alphas.csv')

In [ ]:
# from sklearn import linear_model
# from sklearn.model_selection import KFold
# from sklearn.metrics import log_loss,accuracy_score

# n_splits = 5
# kfold = KFold(n_splits=n_splits,shuffle=True)
# scores = np.zeros([3,2,n_splits]);i=0
# cols = ['elo_diff','s_elo_diff']
# for train_ind,test_ind in kfold.split(df):
#     lm = linear_model.LogisticRegression(fit_intercept = True)
#     train_df,test_df = df.loc[train_ind],df.loc[test_ind]
#     lm.fit(train_df[cols].values.reshape([len(train_df),len(cols)]),train_df['winner'])
#     y_preds = lm.predict(test_df[cols].values.reshape([len(test_df),len(cols)]))
#     y_preds2 = test_df['match_prob_kls']>.5
#     y_preds3 = test_df['match_prob_kls_JS']>.5
#     y_probs = lm.predict_proba(test_df[cols].values.reshape([len(test_df),len(cols)]))
#     scores[0][0][i]=accuracy_score(test_df['winner'],y_preds)
#     scores[0][1][i]=log_loss(test_df['winner'],y_probs,labels=[0,1])
#     scores[1][0][i]=accuracy_score(test_df['winner'],y_preds2)
#     scores[1][1][i]=log_loss(test_df['winner'],test_df['match_prob_kls'],labels=[0,1])
#     scores[2][0][i]=accuracy_score(test_df['winner'],y_preds3)
#     scores[2][1][i]=log_loss(test_df['winner'],test_df['match_prob_kls_JS'],labels=[0,1])

    
#     i+=1
# print '% s_elo used in lm fit: ',lm.coef_[0][1]/(lm.coef_[0][0]+lm.coef_[0][1])
# print 'accuracy: ', np.mean(scores[0][0])
# print 'loss: ', np.mean(scores[0][1])

# print 'kls probabilities'
# print 'accuracy: ', np.mean(scores[1][0])
# print 'loss: ', np.mean(scores[1][1])

# print 'kls JS probabilities'
# print 'accuracy: ', np.mean(scores[2][0])
# print 'loss: ', np.mean(scores[2][1])

In [ ]:
# from sklearn import linear_model
# from sklearn.model_selection import KFold
# from sklearn.metrics import log_loss,accuracy_score

# # cols is a list of all column sets to test; compare with kls pre-match forecasts
# def validate_results(df,columns,n_splits=5):
#     kfold = KFold(n_splits=n_splits,shuffle=True)
#     scores = np.zeros([len(columns)+2,2,n_splits]);i=0
#     for train_ind,test_ind in kfold.split(df):
#         lm = linear_model.LogisticRegression(fit_intercept = True)
#         train_df,test_df = df.loc[train_ind],df.loc[test_ind]
        
#         for k,cols in enumerate(columns):
#             lm.fit(train_df[cols].values.reshape([len(train_df),len(cols)]),train_df['winner'])
#             y_preds = lm.predict(test_df[cols].values.reshape([len(test_df),len(cols)]))
#             y_probs = lm.predict_proba(test_df[cols].values.reshape([len(test_df),len(cols)]))
#             scores[k][0][i]=accuracy_score(test_df['winner'],y_preds)
#             scores[k][1][i]=log_loss(test_df['winner'],y_probs,labels=[0,1])
        
#         y_preds2 = test_df['match_prob_kls']>.5
#         y_preds3 = test_df['match_prob_kls_JS']>.5
#         scores[len(columns)][0][i]=accuracy_score(test_df['winner'],y_preds2)
#         scores[len(columns)][1][i]=log_loss(test_df['winner'],test_df['match_prob_kls'],labels=[0,1])
#         scores[len(columns)+1][0][i]=accuracy_score(test_df['winner'],y_preds3)
#         scores[len(columns)+1][1][i]=log_loss(test_df['winner'],test_df['match_prob_kls_JS'],labels=[0,1])
#         i+=1
    
#     for i,cols in enumerate(columns):
#         print 'columns: ',cols
#         #print '% s_elo used in lm fit: ',lm.coef_[0][1]/(lm.coef_[0][0]+lm.coef_[0][1])
#         print 'accuracy: ', np.mean(scores[i][0])
#         print 'loss: ', np.mean(scores[i][1])
    
#     print 'kls probabilities'
#     print 'accuracy: ', np.mean(scores[len(columns)][0])
#     print 'loss: ', np.mean(scores[len(columns)][1])

#     print 'kls JS probabilities'
#     print 'accuracy: ', np.mean(scores[len(columns)+1][0])
#     print 'loss: ', np.mean(scores[len(columns)+1][1])